In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [3]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot as plt
import pickle

from chemistry import Molecule
from util import score
from pprint import pprint


In [4]:
with open('../data/molecules_enh.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [5]:
structures = pd.read_feather('../data/structures_enh.feather')
molecules_df = pd.read_feather('../data/molecules.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

In [6]:
labelled_enh = labelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#labelled_enh.head(32)

In [7]:
unlabelled_enh = unlabelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#unlabelled_enh.head(32)

In [10]:
from models import SKModel, partition_data

def fn(data_df):
    #data_df = labelled[labelled.type == '1JHC'].head(10)
    data_df = labelled_enh.head(16)
    train_df, valid_df, test_df = partition_data(data_df, train_frac=1)
    train_df = train_df#.copy()

    model = SKModel(molecules=molecules,
                    structures=structures)
    input_df, numeric_df, output_df = model.setup_data(train_df, train_df)

%lprun -f SKModel.make_input fn(labelled_enh.head(10000))

Timer unit: 1e-06 s

Total time: 4.05767 s
File: /home/gautham/work/kaggle/scalar_coupling/notebooks/models.py
Function: make_input at line 166

Line #      Hits         Time  Per Hit   % Time  Line Contents
   166                                               def make_input(self, input_df):
   167         1    1128443.0 1128443.0     27.8          df = self.make_dist(input_df)
   168         1        179.0    179.0      0.0          df['type'] = input_df['type']
   169                                           
   170         4         16.0      4.0      0.0          for t in self.coupling_types:
   171         3       5474.0   1824.7      0.1              df[f'coupling_{t}'] = (df.type == t).astype('int8')
   172                                           
   173         1     570301.0 570301.0     14.1          self.make_atom_columns(df, df.atom_0, df.atom_index_0, 'atom0')
   174         1     569333.0 569333.0     14.0          self.make_atom_columns(df, df.atom_1, df.atom_index_1,